<table align="left">
    <tr>
        <td style="vertical-align: middle; padding-left: 0px; padding-right: 0px;">
            <a href="https://creativecommons.org/licenses/by/4.0/">
                <img src="https://licensebuttons.net/l/by/4.0/80x15.png" />
            </a>
        </td>
        <td style="vertical-align: middle; padding-left: 5px; padding-right: 0px;">
            <a href="https://opensource.org/licenses/MIT">
                <img src="https://img.shields.io/badge/License-MIT-green.svg" />
            </a>
        </td>
        <td style="vertical-align: middle; padding-left: 15px;">
            &copy; Guillaume Rongier
        </td>
    </tr>
</table>

# Source-to-sink simulation

This notebook builds upon the [delta example](https://github.com/badlands-model/badlands-workshop/tree/master/examples/delta) from [Badlands](https://badlands.readthedocs.io/en/latest/index.html) to show how to do source-to-sink simulations with StratigraPy.

### Imports

Let's first import all the required packages and components:

In [ ]:
import numpy as np
from scipy.ndimage import gaussian_filter
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import cmocean

from landlab.components import FlowDirectorD8, FlowDirectorMFD, FlowAccumulator

from stratigrapy import RasterModelGrid
from stratigrapy.components import SeaLevelCalculator, GravityDrivenRouter, WaterDrivenRouter

### Functions

And define a function to generate the initial topography:

In [ ]:
def dome(x, y, center, axis_length, base):
    """
    Generates a dome-like topography.

    Based on https://github.com/badlands-model/badlands-companion/blob/master/badlands_companion/toolGeo.py

    Parameters
    ----------
    x : float or np.ndarray
        Coordinates along the x axis where to compute the elevation.
    y : float or np.ndarray
        Coordinates along the y axis where to compute the elevation.
    center : array-like of shape (2,)
        x and y coordinates of the center of the dome.
    axis_length : array-like of shape (3,)
        Length of the dome along the x, y, and z axes.
    base : float
        Base elevation of the dome.

    Returns
    -------
    elevation : np.ndarray
        Elevation of the dome at the x and y coordinates.
    """
    if isinstance(axis_length, (int, float)):
        axis_length = (axis_length, axis_length, axis_length)

    distance = (x - center[0])**2 / axis_length[0]**2 + (y - center[1])**2 / axis_length[1]**2

    elevation = np.zeros_like(x)
    elevation[distance < 1.] = base + axis_length[2]*np.sqrt(1. - distance[distance < 1.])

    return elevation

## 1. Diffusive model

We'll use the same time setting as before, with a simulation time of 500$\,$000 years and timesteps of 100 years:

In [ ]:
timestep = 100.
runtime = 500000.
n_iterations = int(runtime/timestep)

Then we define the grid:

In [ ]:
grid = RasterModelGrid((49, 49),
                       xy_spacing=(500., 500.),
                       number_of_classes=3,
                       initial_allocation=n_iterations//100 + 100,
                       number_of_layers_to_fuse=100,
                       number_of_top_layers=100)

The initial dome-like topography:

In [ ]:
elevation = grid.add_zeros('topographic__elevation', at='node', clobber=True)
elevation[:] = dome(grid.x_of_node,
                    grid.y_of_node,
                    (12000., 12000.),
                    (4000., 4000., 3000.),
                    -1000.)
elevation[elevation < 0.] = 0.
elevation[:] = gaussian_filter(elevation.reshape(grid.shape), 1.6).ravel()
elevation += dome(grid.x_of_node,
                  grid.y_of_node,
                  (12000., 12000.),
                  (20000., 20000., 3000.),
                  -2980.)

The component controlling sea level:

In [ ]:
slc = SeaLevelCalculator(grid,
                         wavelength=280000.,
                         time_shift=140000.,
                         amplitude=80.)

Finally, we start with the component for gravity-driven diffusion, i.e., where sediments only move based on the slope:

In [ ]:
gdd = GravityDrivenRouter(grid,
                          diffusivity_cont=[6e-2, 4e-2, 2e-2],
                          diffusivity_mar=[4e-1, 2e-1, 1e-1],
                          wave_base=20.,
                          max_erosion_rate_sed=1e-2,
                          max_erosion_rate_br=1e-2,
                          bedrock_composition=[0.55, 0.3, 0.15],
                          fields_to_track='bathymetric__depth')

We can now run the simulation:

In [ ]:
for i in tqdm(range(n_iterations)):
    slc.run_one_step(timestep)
    gdd.run_one_step(timestep)
    grid.stacked_layers.fuse(time=np.mean, bathymetric__depth=np.mean)
grid.stacked_layers.fuse(finalize=True, time=np.mean, bathymetric__depth=np.mean)

And visualize the result:

In [ ]:
fig, ax = plt.subplots()

raster_x = grid.x_of_node[grid.core_nodes].reshape(grid.cell_grid_shape)
raster_y = grid.y_of_node[grid.core_nodes].reshape(grid.cell_grid_shape)
raster_z = grid.at_node['topographic__elevation'][grid.core_nodes].reshape(grid.cell_grid_shape)

pc = ax.pcolormesh(raster_x, raster_y, raster_z, cmap=cmocean.cm.topo,
                   norm=mcolors.CenteredNorm(grid.at_grid['sea_level__elevation']))
fig.colorbar(pc, ax=ax, label='Elevation (m)')

ax.set(xlabel='y (m)', ylabel='y (m)', aspect='equal');

In [ ]:
fig, ax = plt.subplots(figsize=(10, 3))

# Sediments
pc = grid.plot_layers(ax, 'bathymetric__depth', cmap=cmocean.cm.deep, zorder=2)
fig.colorbar(pc[0], ax=ax, label='Water depth (m)')

raster_y = grid.y_of_node[grid.core_nodes].reshape(grid.cell_grid_shape)[:, 23]
raster_z = grid.at_node['topographic__elevation'][grid.core_nodes].reshape(grid.cell_grid_shape)[:, 23]
# Sea level
fill_sea = ax.fill_between(raster_y, raster_z, grid.at_grid['sea_level__elevation'],
                           color='#c6dbef', zorder=0)
# Bedrock
ymin, ymax = ax.get_ylim()
ax.fill_between(raster_y, raster_z, ymin, color='#d9d9d9', zorder=1)

ax.set(xlabel='y (m)', ylabel='z (m)', ylim=(ymin, ymax));

In [ ]:
fig, ax = plt.subplots(figsize=(10, 3))

# Sediments
pc = grid.plot_layers(ax, 'time', cmap='viridis', zorder=2)
fig.colorbar(pc[0], ax=ax, label='Deposition time (yr)')

raster_y = grid.y_of_node[grid.core_nodes].reshape(grid.cell_grid_shape)[:, 23]
raster_z = grid.at_node['topographic__elevation'][grid.core_nodes].reshape(grid.cell_grid_shape)[:, 23]
# Sea level
fill_sea = ax.fill_between(raster_y, raster_z, grid.at_grid['sea_level__elevation'],
                           color='#c6dbef', zorder=0)
# Bedrock
ymin, ymax = ax.get_ylim()
ax.fill_between(raster_y, raster_z, ymin, color='#d9d9d9', zorder=1)

ax.set(xlabel='y (m)', ylabel='z (m)', ylim=(ymin, ymax));

In [ ]:
fig, ax = plt.subplots(figsize=(10, 3))

# Sediments
pc = grid.plot_layers(ax, 'composition', i_class=0, mask_wedges=True, cmap='pink', zorder=2)
fig.colorbar(pc[0], ax=ax, label='Fraction of the first sediment class')

raster_y = grid.y_of_node[grid.core_nodes].reshape(grid.cell_grid_shape)[:, 23]
raster_z = grid.at_node['topographic__elevation'][grid.core_nodes].reshape(grid.cell_grid_shape)[:, 23]
# Sea level
fill_sea = ax.fill_between(raster_y, raster_z, grid.at_grid['sea_level__elevation'],
                           color='#c6dbef', zorder=0)
# Bedrock
ymin, ymax = ax.get_ylim()
ax.fill_between(raster_y, raster_z, ymin, color='#d9d9d9', zorder=1)

ax.set(xlabel='y (m)', ylabel='z (m)', ylim=(ymin, ymax));

## 2. Transport-limited model

Let's do the same but with a water-driven component this time, switching from a linear-diffusion to a transport-limited model for sediment transport. The first steps remain the same, same grid and initial topography:

In [ ]:
grid = RasterModelGrid((49, 49),
                       xy_spacing=(500., 500.),
                       number_of_classes=3,
                       initial_allocation=n_iterations//100 + 100,
                       number_of_layers_to_fuse=100,
                       number_of_top_layers=100)

In [ ]:
elevation = grid.add_zeros('topographic__elevation', at='node', clobber=True)
elevation[:] = dome(grid.x_of_node,
                    grid.y_of_node,
                    (12000., 12000.),
                    (4000., 4000., 3000.),
                    -1000.)
elevation[elevation < 0.] = 0.
elevation[:] = gaussian_filter(elevation.reshape(grid.shape), 1.6).ravel()
elevation += dome(grid.x_of_node,
                  grid.y_of_node,
                  (12000., 12000.),
                  (20000., 20000., 3000.),
                  -2980.)

This time we need to add precipitation as source of water. Here, we use a simple setting, with uniform precipitation of 1 m/yr:

In [ ]:
water_influx = grid.add_ones('water__unit_flux_in', at='node', clobber=True) # m/yr

The component controlling sea level remains the same as before:

In [ ]:
slc = SeaLevelCalculator(grid,
                         wavelength=280000.,
                         time_shift=140000.,
                         amplitude=80.)

We need to add components to compute the discharge:

In [ ]:
fd = FlowDirectorD8(grid)

In [ ]:
fa = FlowAccumulator(grid, flow_director=fd)

And the component for water-driven diffusion, i.e., the transport-limited model:

In [ ]:
wdr = WaterDrivenRouter(grid,
                        transportability_cont=[6e-8, 4e-8, 2e-8],
                        transportability_mar=[4e-7, 2e-7, 1e-7],
                        wave_base=20.,
                        max_erosion_rate_sed=1e-2,
                        max_erosion_rate_br=1e-2,
                        bedrock_composition=[0.55, 0.3, 0.15],
                        fields_to_track='bathymetric__depth')

We can now run the simulation:

In [ ]:
for i in tqdm(range(n_iterations)):
    slc.run_one_step(timestep)
    fa.run_one_step()
    wdr.run_one_step(timestep)
    grid.stacked_layers.fuse(time=np.mean, bathymetric__depth=np.mean)
grid.stacked_layers.fuse(finalize=True, time=np.mean, bathymetric__depth=np.mean)

And visualize the result:

In [ ]:
fig, ax = plt.subplots()

raster_x = grid.x_of_node[grid.core_nodes].reshape(grid.cell_grid_shape)
raster_y = grid.y_of_node[grid.core_nodes].reshape(grid.cell_grid_shape)
raster_z = grid.at_node['topographic__elevation'][grid.core_nodes].reshape(grid.cell_grid_shape)

pc = ax.pcolormesh(raster_x, raster_y, raster_z, cmap=cmocean.cm.topo,
                   norm=mcolors.CenteredNorm(grid.at_grid['sea_level__elevation']))
fig.colorbar(pc, ax=ax, label='Elevation (m)')

ax.set(xlabel='y (m)', ylabel='y (m)', aspect='equal');

In [ ]:
fig, ax = plt.subplots(figsize=(10, 3.75))

# Sediments
pc = grid.plot_layers(ax, 'bathymetric__depth', cmap=cmocean.cm.deep, zorder=2)
fig.colorbar(pc[0], ax=ax, label='Water depth (m)')

raster_y = grid.y_of_node[grid.core_nodes].reshape(grid.cell_grid_shape)[:, 23]
raster_z = grid.at_node['topographic__elevation'][grid.core_nodes].reshape(grid.cell_grid_shape)[:, 23]
# Sea level
fill_sea = ax.fill_between(raster_y, raster_z, grid.at_grid['sea_level__elevation'],
                           color='#c6dbef', zorder=0)
# Bedrock
ymin, ymax = ax.get_ylim()
ax.fill_between(raster_y, raster_z, ymin, color='#d9d9d9', zorder=1)

ax.set(xlabel='y (m)', ylabel='z (m)', ylim=(ymin, ymax));

In [ ]:
fig, ax = plt.subplots(figsize=(10, 3.75))

# Sediments
pc = grid.plot_layers(ax, 'time', cmap='viridis', zorder=2)
fig.colorbar(pc[0], ax=ax, label='Deposition time (yr)')

raster_y = grid.y_of_node[grid.core_nodes].reshape(grid.cell_grid_shape)[:, 23]
raster_z = grid.at_node['topographic__elevation'][grid.core_nodes].reshape(grid.cell_grid_shape)[:, 23]
# Sea level
fill_sea = ax.fill_between(raster_y, raster_z, grid.at_grid['sea_level__elevation'],
                           color='#c6dbef', zorder=0)
# Bedrock
ymin, ymax = ax.get_ylim()
ax.fill_between(raster_y, raster_z, ymin, color='#d9d9d9', zorder=1)

ax.set(xlabel='y (m)', ylabel='z (m)', ylim=(ymin, ymax));

In [ ]:
fig, ax = plt.subplots(figsize=(10, 3.75))

# Sediments
pc = grid.plot_layers(ax, 'composition', i_class=0, mask_wedges=True, cmap='pink', zorder=2)
fig.colorbar(pc[0], ax=ax, label='Fraction of the first sediment class')

raster_y = grid.y_of_node[grid.core_nodes].reshape(grid.cell_grid_shape)[:, 23]
raster_z = grid.at_node['topographic__elevation'][grid.core_nodes].reshape(grid.cell_grid_shape)[:, 23]
# Sea level
fill_sea = ax.fill_between(raster_y, raster_z, grid.at_grid['sea_level__elevation'],
                           color='#c6dbef', zorder=0)
# Bedrock
ymin, ymax = ax.get_ylim()
ax.fill_between(raster_y, raster_z, ymin, color='#d9d9d9', zorder=1)

ax.set(xlabel='y (m)', ylabel='z (m)', ylim=(ymin, ymax));